In [1]:
import requests as req 
from bs4 import BeautifulSoup, CData, Tag
import time
import tqdm
import pickle
import pandas as pd

In [2]:
# pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)

In [3]:
sites = { 
    'Фонтанка.ру' : ['https://www.fontanka.ru/fontanka.rss', ''],                  
    'LENTA.RU' : ['https://lenta.ru/rss', ''],                  
    'TASS' : ['https://tass.ru/rss/v2.xml', ''],
    'ВЕДОМОСТИ' : ['https://www.vedomosti.ru/rss/news', '']           
}

In [4]:
sites

{'Фонтанка.ру': ['https://www.fontanka.ru/fontanka.rss', ''],
 'LENTA.RU': ['https://lenta.ru/rss', ''],
 'TASS': ['https://tass.ru/rss/v2.xml', ''],
 'ВЕДОМОСТИ': ['https://www.vedomosti.ru/rss/news', '']}

In [5]:
def parsing():
    data = { "data" : [] }
        
    for site in range(len(sites)):
        
        url = list(sites.values())[site][0]
                
        resp = req.get(url)

        soup = BeautifulSoup(resp.content, features="xml")

        tag_item = soup.find_all('item')

        for item in tag_item:
            data['data'].append({
                "title": item.title.text,
                "pubDate": item.pubDate.text,
                "site": list(sites.keys())[site],
            })

    return pd.DataFrame(data['data'])

In [6]:
df = parsing()
df.sample(8)

,title,pubDate,site
465,В СПЧ допустили возможность отключения YouTube...,"Wed, 18 Jan 2023 18:43:14 +0300",ВЕДОМОСТИ
423,Макрон заявил о беспрецедентном кризисе в Евро...,"Thu, 19 Jan 2023 12:44:17 +0300",ВЕДОМОСТИ
113,Самолет в США пропал без вести на подлете к аэ...,"Fri, 20 Jan 2023 12:54:57 +0300",LENTA.RU
253,В Подмосковье внедрят единый стандарт работы ж...,"Fri, 20 Jan 2023 14:10:48 +0300",TASS
474,"ВВП Белоруссии в 2022 году сократился на 4,7%","Wed, 18 Jan 2023 17:14:27 +0300",ВЕДОМОСТИ
481,"Евросоюз поставил Украине вооружений на 11,5 м...","Wed, 18 Jan 2023 15:47:48 +0300",ВЕДОМОСТИ
169,Россиянин заказал убийство пасынка за миллион ...,"Fri, 20 Jan 2023 11:34:00 +0300",LENTA.RU
461,«Газпром» подписал с Казахстаном «дорожную кар...,"Wed, 18 Jan 2023 19:43:21 +0300",ВЕДОМОСТИ


In [7]:
topics = pd.DataFrame(['Библиотека', 'Россия', 'Мир', 'Экономика', 'Интернет и СМИ',
       'Спорт', 'Культура', 'Из жизни', 'Силовые структуры',
       'Наука и техника', 'Бывший СССР', 'nan', 'Дом', 'Сочи', 'ЧМ-2014',
       'Путешествия', 'Ценности', 'Легпром', 'Бизнес', 'МедНовости',
       'Оружие', '69-я параллель', 'Культпросвет ', 'Крым'])

In [8]:
model = None

with (open('../../lenta.ru/models/knb_ppl_clf.pkl', 'rb')) as openfile:
    model = pickle.load(openfile)

In [9]:
predicted_topics = model.predict(df['title'])

In [10]:
df_news = pd.concat([pd.Series(predicted_topics), df], ignore_index=True, axis=1)

df_news = df_news.rename(columns={0: "category", 1: "title", 2: "pub_date", 3: "site"})

df_news['parsed_date'] = None
for index, row in enumerate(df_news['pub_date']):
    parsed = time.strptime(row, "%a, %d %b %Y %H:%M:%S %z")
    df_news['parsed_date'][index] = time.strftime('%Y-%m-%d %H:%M:%S', parsed)

df_news.sample(8)

,category,title,pub_date,site,parsed_date
42,Спорт,В Харьковской и Одесской областях произошли вз...,"Fri, 20 Jan 2023 14:28:00 +0300",LENTA.RU,2023-01-20 14:28:00
327,Россия,Новый посол США может прибыть в Россию уже в э...,"Fri, 20 Jan 2023 14:39:40 +0300",ВЕДОМОСТИ,2023-01-20 14:39:40
517,Мир,"Глава МИД ЮАР заявила, что БРИКС ищет замену д...","Wed, 18 Jan 2023 10:36:00 +0300",ВЕДОМОСТИ,2023-01-18 10:36:00
85,Интернет и СМИ,Власти Украины заявили об обещании Meta не бло...,"Fri, 20 Jan 2023 13:39:00 +0300",LENTA.RU,2023-01-20 13:39:00
439,Россия,В Белгородской области сообщили об обстреле не...,"Thu, 19 Jan 2023 10:27:48 +0300",ВЕДОМОСТИ,2023-01-19 10:27:48
411,Наука и техника,Борисов ожидает завершения строительства старт...,"Thu, 19 Jan 2023 14:15:50 +0300",ВЕДОМОСТИ,2023-01-19 14:15:50
403,Наука и техника,В Минцифры сообщили о льготах для мобильных оп...,"Thu, 19 Jan 2023 15:02:37 +0300",ВЕДОМОСТИ,2023-01-19 15:02:37
87,Мир,В Чехии объяснили срочное наращивание оружия в ЕС,"Fri, 20 Jan 2023 13:37:00 +0300",LENTA.RU,2023-01-20 13:37:00


In [11]:
import json
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')

def write_to_kafka(df):
    hm = {}
    for _, row in df.iterrows():
        hm["category"] = row["category"]
        hm["title"] = row["title"]
        hm["site"] = row["site"]
        hm["pub_date"] = row["parsed_date"]        
        producer.send('foobar', json.dumps(hm).encode('utf-8'))
        producer.flush()

write_to_kafka(df_news)